In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv
import os
import random
import string
import seaborn as sns
import math
import re
import warnings
import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from datetime import datetime
from dateutil.parser import parse
from sklearn import metrics
from unidecode import unidecode
from sklearn.metrics import roc_auc_score

from itertools import combinations

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

%matplotlib inline

In [107]:
data_version_input = "01-10-2020"

In [108]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

In [109]:

f =  "currentLocationName"
vc1 = sorted(df_train[f].value_counts().index)
print(vc1)
print(len(vc1))
print("#####################")
vc2 = sorted(df_test[f].value_counts().index)
print(vc2)
print(len(vc2))
a = []
for i in vc2:
    if i not in vc1:
        a.append(i)
print(len(a))
print(a)


['A Luoi (1), Binh Tri Thien, Vietnam', 'Abu Dhabi, United Arab Emirates', 'Ajaccio', 'American Falls, Idaho', 'Amherst, Massachusetts', 'An Biên', 'An Duong, Hải Phòng, Vietnam', 'An Giang (4), Nghia Binh, Vietnam', 'An Khê, Gia Lai, Vietnam', 'An Lac, Thái Bình, Vietnam', 'An Lão, Bình Ðịnh, Vietnam', 'An Lê, Thái Bình, Vietnam', 'An Nhơn', 'An Ninh, Long An, Vietnam', 'An Ninh, Quảng Ninh (Quảng Bình)', 'An Phong, Ðồng Tháp, Vietnam', 'An Phu', 'An Phú (thị trấn)', 'An Thuan, Song Be, Vietnam', 'Angiang, An Giang, Vietnam', 'Anh Son', 'Anh Sơn', 'Anini-y, Antique', 'Ap An Thai (1), Tiền Giang, Vietnam', 'Ap Bao Binh, Vietnam', 'Ap Ben Cam, Vietnam', 'Ap Ben Cau, Tây Ninh, Vietnam', 'Ap Binh Duong (1), Vietnam', 'Ap Binh Hung, Tiền Giang, Vietnam', 'Ap Binh Quang, Long An, Vietnam', 'Ap Binh Thuan, Minh Hai, Vietnam', 'Ap Hung Yen, Song Be, Vietnam', 'Ap Lam Phu, Lâm Ðồng, Vietnam', 'Ap Loi Nhon, Tiền Giang, Vietnam', 'Ap My Tho, Tiền Giang, Vietnam', 'Ap Nhon My, An Giang, Vietnam',

In [110]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)

In [111]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G", "Field_3", "Field_38"],
    
    "cat_str" : ["Field_4", "Field_12","Field_36", "Field_54", "Field_62","Field_65","Field_66",\
                 "info_social_sex", "brief", "gioiTinh","currentLocationState", "Field_61", \
                 "currentLocationCountry", 'homeTownState', 'data.basic_info.locale',\
                "homeTownCountry", "Field_55"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh", "Field_34"],
    
    "address" : ["Field_45", "Field_48", "Field_49", "Field_56", 'homeTownName', "currentLocationCity", \
                 "homeTownCity", "diaChi", "currentLocationName"],
    
    "drop" : ["maCv", "Field_18", "Field_68", "Field_46"]
}

In [112]:
lst = data_type["cat_num"] + data_type["cat_str"] + data_type["date"] + data_type["address"] + data_type["drop"]

In [113]:
"""
for f in df_transform.columns:
    vc = df_transform[f].value_counts()
    print(f"{f} : {len(vc)} : {vc.index[:10]}")
"""

'\nfor f in df_transform.columns:\n    vc = df_transform[f].value_counts()\n    print(f"{f} : {len(vc)} : {vc.index[:10]}")\n'

consider transform: "brief" \
consider drop: "Field_56", "maCv"


In [114]:
df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["address"], axis = 1)

In [115]:
def fix38(x):
    if x == 0.0 or x == "0" or x == "0.0":
        return 0
    if x == 1.0 or x == "1":
        return 1
    return np.nan

In [116]:
def fix54(x):
    if x == "1" or x == "01" or x == ' P. Tân Phú':
        return "VN"
    if x in ['IT', 'GB', 'SE', 'TK', 'BD']:
        return np.nan
    return x

In [117]:
def fix55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [118]:
def fix61(x):
    if x == 'Chất độc hóa học':
        return np.nan
    return x

In [119]:
def fixcurrentLocationState(x):
    if x in ['Ilhas', 'Daegu', 'Auckland Region', 'Basse-Normandie', 'Chongqing', "Provence-Alpes-Côte d'Azur",\
             'Rajshahi Division', 'Hubei', 'Mandalay', 'Missouri', 'Pathum Thani', 'Central Java', 'New Hampshire',\
             'Odisha', 'South Australia', 'Chon Buri', 'Haute-Normandie', 'Kirov Oblast', 'Primorsky Krai', 'Arizona',\
             'Ulsan', 'Kanagawa Prefecture', 'Metro Manila', 'Kachin State', 'Svay Rieng Province', 'Nagano Prefecture',\
             'Shiga Prefecture', 'Oslo', 'Indiana', 'Hunan', 'Bolikhamsai Province', 'Krasnoyarsk Krai',\
             'Distrito Nacional', 'Chiba Prefecture']:
        return np.nan
    return x

In [120]:
def fixcurrentLocationCountry(x):
    if x in ['Macau', 'New Zealand', 'Dominican Republic']:
        return np.nan
    return x

In [121]:
def fixhomeTownState(x):
    if x in ['Hunan', 'Kachin State', 'Distrito Federal', 'Bizerte Governorate', 'Central Java',\
             'Svay Rieng Province', 'Distrito Nacional', 'County Mayo', 'Odisha', 'Lazio', 'Nevada',\
             'Daegu', 'Arizona', 'Equatoria', 'Fukuoka Prefecture', 'Taoyuan City', 'Kandal Province', \
             'State of Mexico', 'Hebei', 'New South Wales', 'Capital Governorate', 'Oregon']:
        return np.nan
    return x

In [122]:
def fixhomeTownCountry(x):
    if x in ['Australia', 'Bahrain', 'Dominican Republic', 'Ireland', 'South Sudan', 'Tunisia']:
        return np.nan
    return x

In [123]:
def fixdata_basic_info_locale(x):
    if x == "_VN":
        return 'vi_VN'
    elif x in ['lv_LV', 'ru_RU']:
        return np.nan
    return x

In [124]:
def calculate_age(born):
    born = datetime.strptime(born,'%Y-%m-%d').date()
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [125]:
def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    return df

In [126]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [127]:
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [128]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        df[col] = df[col].apply(date_normalize)

    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%m-%Y')
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_startDate'] = df[f'{cat}_startDate'].dt.strftime('%m-%Y')
        df[f'{cat}_endDate'] = df[f'{cat}_endDate'].dt.strftime('%m-%Y')
        
        cat_cols.append(f'{cat}_startDate')
        cat_cols.append(f'{cat}_endDate')
    
    for col in cat_cols:
        df[col] = df[col]
        
    return df

In [129]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize)
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize)

    return df

In [130]:
df_transform["Field_38"] = df_transform["Field_38"].apply(fix38)
df_transform["Field_54"] = df_transform["Field_54"].apply(fix54)
df_transform["Field_55"] = df_transform["Field_55"].apply(fix55)
df_transform["Field_61"] = df_transform["Field_61"].apply(fix61)

df_transform["currentLocationState"] = df_transform["currentLocationState"].apply(fixcurrentLocationState)
df_transform["currentLocationCountry"] = df_transform["currentLocationCountry"].apply(fixcurrentLocationCountry)
df_transform["homeTownState"] = df_transform["homeTownState"].apply(fixhomeTownState)
df_transform["homeTownCountry"] = df_transform["homeTownCountry"].apply(fixhomeTownCountry)
df_transform['data.basic_info.locale'] = df_transform['data.basic_info.locale'].apply(fixdata_basic_info_locale)
df_transform["age"] = df_transform["ngaySinh"].map(lambda x: calculate_age(x) if x is not np.nan else x)
df_transform["null_sum"] = df_transform.isnull().sum(axis=1)

df_transform = process_gender(df_transform)
df_transform = process_datetime_cols(df_transform)

df_transform = process_location(df_transform)

In [131]:
drop = ["gioiTinh", "info_social_sex", "ngaySinh"]
df_transform = df_transform.drop(drop, axis = 1)

In [132]:
categoricals = []
for header in df_transform.columns:
    if str(df_transform[header].dtypes) is "object":
        categoricals.append(header)
for col in categoricals:
    df_transform[col] = df_transform[col].astype(str)
lbl = LabelEncoder()
for col in categoricals:
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    df_transform[col] = df_transform[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(df_transform[col]))
    
    # Take if vals appeared both 5 times
    common_vals = set(df_transform[col].value_counts()[df_transform[col].value_counts()> 100].index)
    
    # Replace not-common values with "Missing" or np.NaN      
    df_transform.loc[~df_transform[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(df_transform[col].tolist())
    df_transform[col] = lbl.transform(df_transform[col])


In [133]:
fill_1 = []
for header in df_transform.columns:
    vc = df_transform[header].value_counts()
    if len(vc) < 2:
        fill_1.append(header)
for col in fill_1:
    df_transform[col] = df_transform[col].fillna(-1)

In [134]:
df_transform = df_transform.fillna(df_transform.mean())

In [135]:
import re
df_transform = df_transform.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [136]:
scaler = MinMaxScaler()
scaler.fit(df_transform)
df_transform = scaler.transform(df_transform)
df_transform = pd.DataFrame(df_transform)

In [137]:
df_transform

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133
0,0.500000,0.529412,0.000000,0.0,0.909091,0.916667,0.142857,0.446809,0.911765,0.200000,0.518519,0.333333,1.0,1.000000,0.000000,0.142906,0.75000,0.500000,0.034483,0.914286,0.000000,0.000000,0.000000,0.911765,0.690141,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.8125,0.0,0.593629,0.096207,0.551133,0.512462,0.000000,0.252174,0.305970,0.5
1,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.4375,0.0,0.593629,0.096207,0.551133,0.512462,0.000000,0.270380,0.731343,1.0
2,0.058824,0.058824,1.000000,0.5,0.060606,0.055556,0.142857,0.936170,0.058824,0.200000,1.000000,1.000000,1.0,0.058824,0.000000,0.167785,0.75000,0.500000,0.021552,0.057143,0.000000,0.000000,0.000000,0.058824,0.239437,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0625,0.0,0.616514,0.029922,0.572027,0.582456,0.333333,0.191304,0.201493,0.0
3,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,1.000000,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.0625,0.0,0.616514,0.029922,0.572027,0.582456,0.666667,0.270380,0.955224,1.0
4,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,1.000000,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.0625,0.0,0.616514,0.029922,0.572027,0.582456,0.666667,0.270380,0.955224,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73406,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,1.000000,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.0625,0.0,0.600000,0.118485,0.540255,0.301754,0.666667,0.270380,0.955224,1.0
73407,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,1.000000,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.1875,0.0,0.636697,0.038176,0.572204,0.582456,0.666667,0.270380,0.955224,1.0
73408,0.911765,0.911765,0.000000,0.0,0.909091,0.916667,0.142857,0.957447,0.911765,0.200000,1.000000,1.000000,1.0,1.000000,0.585255,0.206800,0.75000,0.500000,0.370690,0.914286,0.000000,0.000000,0.333333,0.911765,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.9375,0.0,0.593629,0.096207,0.551133,0.512462,0.000000,0.330435,0.194030,0.0
73409,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.7477

In [138]:
train_dev = pd.concat([df_train.id, df_train.label, df_transform.iloc[:len(df_train),:]], axis = 1)
test = df_transform.iloc[len(df_train):,:].reset_index(drop=True)
test = pd.concat([df_test.id, test], axis = 1)

In [139]:
test

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,...,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133
0,53030,0.735294,0.764706,1.000000,0.5,0.515152,0.555556,0.685714,0.595745,0.529412,0.200000,1.000000,1.000000,1.0,0.529412,0.000000,0.050000,0.75000,0.500000,0.092672,0.542857,0.000000,0.000000,0.166667,0.529412,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.9375,0.25,0.593629,0.096207,0.551133,0.512462,0.000000,0.217391,0.283582,0.5
1,53031,0.647059,0.676471,1.000000,0.5,0.575758,0.611111,0.685714,0.595745,0.588235,0.200000,1.000000,1.000000,1.0,1.000000,0.000000,0.035000,0.75000,0.500000,0.051724,0.600000,0.000000,0.000000,0.166667,0.588235,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.0000,0.00,0.593629,0.096207,0.551133,0.512462,0.000000,0.234783,0.268657,0.5
2,53032,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.593629,0.096207,0.551133,0.512462,0.000000,0.270380,0.798507,1.0
3,53033,0.735294,0.764706,1.000000,0.5,0.666667,0.694444,0.857143,0.765957,0.676471,0.200000,1.000000,1.000000,1.0,0.676471,0.000000,0.050000,0.75000,0.500000,0.000000,0.685714,0.000000,0.000000,0.166667,0.676471,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.9375,0.00,0.593629,0.096207,0.551133,0.512462,0.000000,0.200000,0.261194,0.0
4,53034,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.3125,0.00,0.605505,0.096207,0.609982,0.629825,0.000000,0.270380,0.522388,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.0625,0.00,0.600000,0.118485,0.540255,0.301754,0.666667,0.270380,0.955224,1.0
20377,73407,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,1.000000,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.1875,0.00,0.636697,0.038176,0.572204,0.582456,0.666667,0.270380,0.955224,1.0
20378,73408,0.911765,0.911765,0.000000,0.0,0.909091,0.916667,0.142857,0.957447,0.911765,0.200000,1.000000,1.000000,1.0,1.000000,0.585255,0.206800,0.75000,0.500000,0.370690,0.914286,0.000000,0.000000,0.333333,0.911765,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.9375,0.00,0.593629,0.096207,0.551133,0.512462,0.000000,0.330435,0.194030,0.0
20379,73409,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726

In [140]:
train_dev

,id,label,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,...,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133
0,0,1,0.500000,0.529412,0.000000,0.0,0.909091,0.916667,0.142857,0.446809,0.911765,0.200000,0.518519,0.333333,1.0,1.000000,0.000000,0.142906,0.75000,0.500000,0.034483,0.914286,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.8125,0.0,0.593629,0.096207,0.551133,0.512462,0.000000,0.252174,0.305970,0.5
1,1,0,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.4375,0.0,0.593629,0.096207,0.551133,0.512462,0.000000,0.270380,0.731343,1.0
2,2,0,0.058824,0.058824,1.000000,0.5,0.060606,0.055556,0.142857,0.936170,0.058824,0.200000,1.000000,1.000000,1.0,0.058824,0.000000,0.167785,0.75000,0.500000,0.021552,0.057143,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0625,0.0,0.616514,0.029922,0.572027,0.582456,0.333333,0.191304,0.201493,0.0
3,3,0,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.0625,0.0,0.616514,0.029922,0.572027,0.582456,0.666667,0.270380,0.955224,1.0
4,4,1,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,0.0625,0.0,0.616514,0.029922,0.572027,0.582456,0.666667,0.270380,0.955224,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53025,53025,0,0.941176,0.941176,0.000000,0.0,0.939394,0.944444,0.171429,0.957447,0.941176,0.200000,1.000000,1.000000,1.0,1.000000,0.141503,0.050000,0.75000,0.500000,0.010776,0.942857,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0625,0.0,0.926606,0.438538,0.585952,0.261404,0.000000,0.278261,0.104478,0.0
53026,53026,0,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.0,0.603670,0.028470,0.583179,0.585965,0.000000,0.270380,0.597015,0.0
53027,53027,0,0.764706,0.794118,1.000000,0.5,0.969697,0.416667,0.514286,0.702128,0.970588,0.200000,0.962963,0.333333,1.0,0.970588,0.000000,0.134732,0.75000,0.500000,0.000000,0.971429,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.0,0.593629,0.096207,0.551133,0.512462,0.000000,0.269565,0.223881,0.0
53028,53028,0,1.000000,1.000000,0.535958,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.219541,1.000000,1.000000,0.0,1.000000,0.039078,0.121453,0.74774,0.499726,0.068528,1.000000,0.010755,0.010615,0.091182,...,0.000000,0.000000,0.000000,0.

In [142]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': 1,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.7,
    'min_data_in_leaf': 200,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'min_hessian': 0.01,
    'feature_fraction_seed': 2,
    'bagging_seed': 3
}

In [143]:
def compute_gini(labels, preds):
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    gini = 2 * auc - 1
    return gini

In [144]:
iter_predicttion = []
for i in range(1):
    pred_stack = []
    kf = KFold(n_splits = 4, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,2:]
        dev1 = train_dev.iloc[dev_index,2:]

        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_test = clf.predict(test.iloc[:,1:])
        predictions_dev = clf.predict(dev1)
        pred_stack.append(predictions_test)
        print(compute_gini(train_dev.loc[dev_index]["label"], predictions_dev))
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)

    iter_predicttion.append(predictions)

0.48231156084460736
0.47451732012670056
0.468484343087058
0.47302985297507516


In [145]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)

In [147]:
data_version = "10-10-2020-c"

In [148]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)